## CNN
Welcome to second part of tutorial. <br>
CNNs are Convolutional Neural Networks, Due to limitations of MLP CNNs were born. <br>
I'm sure you guys already have looked at lectures. Now it's time to play. <br>
I would like to remind you. It's very important that you develop understanding of **MLP** and **CNNs** as they will be primary building blocks of DRL

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x) # convolution layer 1
        x = F.relu(x) # activation function
        x = self.conv2(x) # convolution layer 2
        x = F.relu(x) # activation function
        x = F.max_pool2d(x, 2) # polling layer
        x = self.dropout1(x) # drop out
        x = torch.flatten(x, 1) # flatten
        x = self.fc1(x) # fully connected
        x = F.relu(x) # activation function
        x = self.dropout2(x) # drop out
        x = self.fc2(x) # fully connected
        output = F.log_softmax(x, dim=1)  
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval):
    """
    Function to train the model.
    
    Parameters
    ----------
    model: instance of model.
    
    device: GPU/CPU
        determined by pytorch if you have cuda installed. by default CPU
        
    train_loader: DataLoader
        instance to load training data, useful to form batches and do data transformation.
    
    optimiser: optimiser
    
    epoch: int
    
    log_interval: int
        the interval with which you want to log your metrices.
    
    Returns
    -------
    train_loss: float
    
    train_accuracy: float
    """
    model.train()
    correct = 0
    train_loss = list()
    train_accuracy = list()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            accuracy =100. * correct / len(train_loader.dataset)
            train_accuracy.append(accuracy)
            train_loss.append(loss.item())
            print('Train Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader)))
    return train_loss, train_accuracy


def test(model, device, test_loader):
    """
    Function to test the model.
    
    Parameters
    ----------
    model: instance of model.
    
    device: GPU/CPU
        determined by pytorch if you have cuda installed. by default CPU
        
    test_loader: DataLoader
        instance to load testing data, useful to form batches and do data transformation.
    
    Returns
    -------
    test_loss: float
    
    test_accuracy: float
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    return test_loss, accuracy


In [4]:
def main():
    # Training settings
    batch_size=64
    test_batch_size=1000
    epochs=10
    lr=0.2
    gamma=0.7
    no_cuda=False
    seed=1
    log_interval=1
    save_model=False
    test_loss_array = list()
    train_loss_array = list()
    test_accuracy_array = list()
    train_accuracy_array = list()
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)
    
    # set device for training (cpu/gpu)
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    # wrap data in class and apply transformations
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)
    
    # declare model and copy it's instance to device
    model = Net().to(device)
    
    # declare optimiser.
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    # 
    for epoch in range(1, epochs + 1):
        train_loss , train_accuracy = train(model, device, train_loader, optimizer, epoch, log_interval)
        test_loss, test_accuracy = test(model, device, test_loader)
        test_loss_array.append(test_loss)
        test_accuracy_array.append(test_accuracy)
        train_loss_array.append(np.mean(train_loss))
        train_accuracy_array.append(np.mean(train_accuracy))
        scheduler.step()

    epoch_count = range(1, len(test_loss_array) + 1)
    # Visualize loss history
    plt.plot(epoch_count, test_loss_array, 'r--')
    plt.plot(epoch_count, train_loss_array, 'b-')
    plt.legend(['Test Loss', 'Train Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show();
    
    plt.plot(epoch_count, test_accuracy_array, 'r--')
    plt.plot(epoch_count, train_accuracy_array, 'b-')
    plt.legend(['Test Accuracy', 'Train Accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show();
    
    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")

In [ ]:
main()